In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
import numpy as np
import cv2
import ctypes
import _ctypes
import sys

kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth | PyKinectV2.FrameSourceTypes_Body)
                
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        if kinect.has_new_color_frame() & kinect.has_new_depth_frame():
            image_np = kinect.get_last_color_frame()
            image_np = np.reshape(image_np, (kinect.color_frame_desc.Height, kinect.color_frame_desc.Width, 4))
            frameC = kinect._color_frame_data
            image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
            
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
            coordinates = vis_util.return_coordinates(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8,
                        min_score_thresh=0.80)

            if coordinates is not None:
            
                #print(*coordinates)
                print(coordinates)
                xpoint_for_depth=[]
                ypoint_for_depth=[]
                
                print("object 개수:", len(coordinates))
                
                # Connect Color and Depth Info.
                frame = kinect.get_last_depth_frame()
                frameD = kinect._depth_frame_data
                frame = frame.astype(np.uint8)
                frame = np.reshape(frame, (424, 512))
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

                # Take just one point : middle
                '''for i in range(len(coordinates)):

                    ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                    xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))
                    
                print("ypoint_for_depth:", ypoint_for_depth)
                print("xpoint_for_depth:", xpoint_for_depth)
                
                ##output = cv2.bilateralFilter(output, 1, 150, 75)
                cv2.imshow('KINECT Video Stream', frame)
            
                for i in range(len(coordinates)):
                    Pixel_Depth = frameD[((ypoint_for_depth[i] * 512) + xpoint_for_depth[i])]
                    print("점 하나만 고려:",i+1, "번째 object의 depth value: ", Pixel_Depth)'''
                    
                # Get average depth value considering all coordinate pairs. (List Version)
                '''Pixel_Depth_List = []
                Pixel_Depth_Sum = 0
                non_error_count = 0
                for i in range(len(coordinates)):
                    for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                        for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                            Pixel_Depth_Sum += frameD[((y * 512) + x)]
                            if frameD[((y * 512) + x)] != 0:
                                non_error_count += 1
                    
                    Pixel_Depth_List.append(int(Pixel_Depth_Sum/non_error_count))
                    
                for i in range(len(coordinates)):
                    print("전체 면적 고려:", i+1, "번째 object의 depth value: ", Pixel_Depth_List[i])
                print("\n")'''
                
                # Get average depth value considering all coordinate pairs. (Dict Version)
                Pixel_Depth_Dict = {}
                Pixel_Depth_Sum = 0
                non_error_count = 0
                for i in range(len(coordinates)):
                    for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                        for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                            Pixel_Depth_Sum += frameD[((y * 512) + x)]
                            if frameD[((y * 512) + x)] != 0:
                                non_error_count += 1
                    
                    Pixel_Depth_Dict[coordinates[i][0]] = int(Pixel_Depth_Sum/non_error_count)
                
                # Compare each objects' depth. (Sorting: close -> far)
                sorted(Pixel_Depth_Dict.items(), key=lambda t : t[1])
                print(Pixel_Depth_Dict)
                print("\n")
                
                # VideoFileOutput.write(image_np)
                cv2.imshow('KINECT Color', image_np)
                cv2.waitKey(1)
                

        key = cv2.waitKey(1)
        if key == 27: break


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['person', 532, 865, 956, 1220, 87.12053298950195]]
object 개수: 1
{'person': 31295}


[['person', 521, 869, 958, 1222, 89.764404296875]]
object 개수: 1
{'person': 31377}


[['person', 521, 867, 958, 1227, 89.34328556060791]]
object 개수: 1
{'person': 31326}


[['person', 521, 866, 959, 1227, 89.25570249557495]]
object 개수: 1
{'person': 31342}


[['person', 524, 870, 964, 1232, 85.21506786346436]]
object 개수: 1
{'person': 31296}


[['person', 520, 868, 962,

[['person', 8, 1069, 836, 1910, 90.13837575912476], ['book', 493, 588, 174, 202, 80.33461570739746]]
object 개수: 2
{'person': 23429, 'book': 23453}


[['person', 1, 1066, 850, 1909, 88.41978311538696], ['book', 492, 590, 173, 202, 81.42379522323608]]
object 개수: 2
{'person': 23237, 'book': 23263}


[['person', 6, 1069, 822, 1908, 88.27880620956421]]
object 개수: 1
{'person': 23069}


[['person', 5, 1072, 819, 1910, 89.50029611587524], ['book', 493, 590, 174, 203, 82.4408769607544]]
object 개수: 2
{'person': 23209, 'book': 23234}


[['person', 11, 1072, 830, 1904, 89.18982744216919], ['book', 493, 590, 175, 202, 81.72187209129333]]
object 개수: 2
{'person': 23163, 'book': 23186}


[['person', 37, 1060, 879, 1918, 91.31155610084534], ['book', 493, 588, 174, 202, 82.61947631835938]]
object 개수: 2
{'person': 23567, 'book': 23591}


[['person', 8, 1072, 824, 1907, 88.13732266426086], ['book', 493, 590, 175, 202, 82.41554498672485]]
object 개수: 2
{'person': 23062, 'book': 23086}


[['person', 8, 1070,

[['person', 5, 1074, 821, 1909, 87.0057463645935], ['book', 494, 590, 175, 203, 80.77925443649292]]
object 개수: 2
{'person': 22890, 'book': 22914}


[['person', 7, 1073, 830, 1913, 89.68313932418823]]
object 개수: 1
{'person': 23343}


[['person', 3, 1072, 823, 1910, 88.25528025627136], ['book', 493, 589, 175, 203, 80.99136352539062]]
object 개수: 2
{'person': 23090, 'book': 23114}


[['person', 9, 1072, 848, 1912, 90.73602557182312], ['book', 494, 588, 174, 202, 82.08312392234802]]
object 개수: 2
{'person': 23269, 'book': 23293}


[['person', 5, 1070, 821, 1909, 87.93314695358276], ['book', 493, 590, 174, 202, 80.39278984069824]]
object 개수: 2
{'person': 22976, 'book': 23001}


[['person', 4, 1073, 820, 1910, 88.64728212356567], ['book', 494, 589, 175, 203, 81.88685178756714]]
object 개수: 2
{'person': 23051, 'book': 23075}


[['person', 5, 1072, 823, 1910, 88.21747899055481]]
object 개수: 1
{'person': 23188}


[['person', 7, 1067, 832, 1909, 89.11975026130676], ['book', 493, 590, 175, 203, 81.39

[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['person', 184, 964, 1034, 1783, 90.83683490753174], ['book', 491, 591, 174, 203, 80.65657615661621]]
object 개수: 2
{'person': 25499, 'book': 25534}


[['person', 206, 954, 1088, 1833, 93.79034042358398]]
object 개수: 1
{'person': 26399}


[['person', 203, 954, 1094, 1843, 91.88687801361084], ['book', 492, 591, 175, 203, 80.63872456550598]]
object 개수: 2
{'person': 26246, 'book': 26278}


[['person', 206, 957, 1088, 1819, 94.86347436904907]]
object 개수: 1
{'person': 26330}


[['person', 203, 955, 1085, 1822, 94.3177580833435]]
object 개수: 1
{'person': 26216}


[['person', 202, 954, 1089, 1834, 93.08249950408936], ['book', 492, 591, 174, 202, 82.67102241516113]]
object 개수: 2
{'person': 26140, 'book': 26172}


[['person', 201, 953, 1097, 1824, 95.77266573905945]]
object 개수: 1
{'person': 26109}


[['person', 203, 955, 1098, 1829, 95.37917375564575]]
object 개수: 1
{'person': 26188}


[['person', 213, 952, 1083, 1804, 95.1045274734497]]
object 개수: 1
{'per

[['person', 193, 929, 920, 1392, 89.86367583274841]]
object 개수: 1
{'person': 26351}


[['person', 189, 936, 923, 1393, 90.83727598190308]]
object 개수: 1
{'person': 26256}


[['person', 183, 931, 922, 1384, 92.24428534507751]]
object 개수: 1
{'person': 26043}


[['person', 181, 933, 922, 1386, 91.56168699264526]]
object 개수: 1
{'person': 25941}


[['person', 179, 931, 919, 1386, 90.46701788902283]]
object 개수: 1
{'person': 25891}


[['person', 181, 940, 919, 1387, 91.08449220657349]]
object 개수: 1
{'person': 25988}


[['person', 183, 941, 921, 1389, 90.4391348361969]]
object 개수: 1
{'person': 26043}


[['person', 181, 940, 926, 1383, 92.04623699188232]]
object 개수: 1
{'person': 25902}


[['person', 185, 940, 928, 1386, 84.85679626464844]]
object 개수: 1
{'person': 26090}


[['person', 187, 942, 925, 1383, 87.563556432724]]
object 개수: 1
{'person': 26157}


[['person', 184, 945, 919, 1404, 88.34565877914429]]
object 개수: 1
{'person': 26099}


[['person', 187, 934, 940, 1437, 92.974454164505]]
object

{'person': 24553}


[['person', 194, 852, 875, 1425, 91.09019041061401]]
object 개수: 1
{'person': 24567}


[]
object 개수: 0
{}


[['person', 212, 897, 844, 1535, 82.6352596282959]]
object 개수: 1
{'person': 25908}


[['person', 232, 908, 857, 1539, 92.76126623153687]]
object 개수: 1
{'person': 26979}


[['person', 258, 944, 880, 1555, 90.8610999584198]]
object 개수: 1
{'person': 28438}


[['person', 243, 916, 865, 1522, 87.77496814727783]]
object 개수: 1
{'person': 28029}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[]
object 개수: 0
{}


[['person', 113, 1061, 430, 1549, 84.7711443901062]]
object 개수: 1
{'person': 25646}


[['person', 19, 1069, 189, 1650, 90.66424369812012]]
object 개수: 1
{'person': 22647}


[]
object 개수: 0
{}


[['person', 3, 1072, 107, 1769, 81.29105567932129]]
object 개수: 1
{'person': 22738}


[['person', 8, 1068, 119, 1771, 84.88511443138123]]
object 개수: 1
{'person': 22907}


[['person', 4, 1071, 106, 1823, 82.84938335418701]]
object 개수: 

KeyboardInterrupt: 